## Setting up

In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
import wandb
wandb.init(mode="disabled")



In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## Loading and processing the dataset

In [ ]:
# df = pd.read_csv("/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv",index_col = "Unnamed: 0") s
# df.loc[:,'status'] = df.loc[:,'status'].str.replace('Bi-Polar','Bipolar')
# df = df[(df.status != "Personality disorder") & (df.status != "Stress") & (df.status != "Suicidal")]
# df.head()

In [17]:
sampled_df = pd.read_json('/kaggle/input/coling-25-task-1/en_dev.jsonl', lines=True)
percentage = 0.022 

# Sample x percent of the DataFrame
df = sampled_df.sample(frac=percentage, random_state=42)  # random_state for reproducibility

df

,id,source,sub_source,lang,model,label,text
80897,39ab45d8-9469-4fb7-a9db-f8a09f4b0658,m4gt,wikihow,en,davinci,1,How to Administer IV Fluids\nWhether it's your...
142010,26013593-265e-4a9d-a4c8-2f3cf03e99f1,mage,tldr,en,text-davinci-002,1,Apple's HomePod mini is a small and mighty sma...
191985,c4604acd-de24-40bb-9286-e41675a77d0b,mage,yelp,en,flan_t5_small,1,This place has been a personal family traditio...
47240,b5ce42f9-2bf9-4f35-9757-7f61b14c6642,mage,wp,en,human,0,"Simply put, my soul is conflicted. My beautifu..."
182695,0be9fe79-d97d-4ded-a690-cededede8e14,m4gt,wikipedia,en,llama3-8b,1,The Legend of the Golden Gun is a mythical tr...
...,...,...,...,...,...,...,...
204200,fde5d596-f619-494d-92b2-e84e0db5e98d,mage,sci_gen,en,opt_30b,1,Recent GAN-based architectures have been able ...
22586,5c3ef9cd-17f9-4254-b5c4-c50d98400514,mage,roct,en,human,0,Xander got up in the morning before everyone e...
96410,5f3d011e-631e-4eeb-84af-b20c6b827052,mage,roct,en,text-davinci-003,1,Once upon a time there was a town called Palom...
152037,999c7db3-dcd5-4868-82dc-a4a96287e4ae,mage,hswag,en,opt_iml_30b,1,It's the same with domesticated rats. Instinct...


In [18]:
# Function to convert labels
def convert_label(label):
    return "human" if label == 0 else "machine"

# Apply the conversion function to the 'label' column
df['label'] = df['label'].apply(convert_label)

# Display the resulting DataFrame
df = df[['text','label']]
df.rename(columns={'text': 'statement', 'label': 'status'}, inplace=True)
df

/tmp/ipykernel_30/1625676964.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'text': 'statement', 'label': 'status'}, inplace=True)


,statement,status
80897,How to Administer IV Fluids\nWhether it's your...,machine
142010,Apple's HomePod mini is a small and mighty sma...,machine
191985,This place has been a personal family traditio...,machine
47240,"Simply put, my soul is conflicted. My beautifu...",human
182695,The Legend of the Golden Gun is a mythical tr...,machine
...,...,...
204200,Recent GAN-based architectures have been able ...,machine
22586,Xander got up in the morning before everyone e...,human
96410,Once upon a time there was a town called Palom...,machine
152037,It's the same with domesticated rats. Instinct...,machine


In [19]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=85).reset_index(drop=True)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]


System_message="You are a highly skilled AI model specialized in detecting whether a given text is machine-generated or human-written. Your task is to analyze the text provided below in English and determine its origin."

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
    {System_message}
    Please classify the following text and provide your answer as either "machine generated" or "human written".

    Text: {data_point["statement"]}
    label: {data_point["status"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
    {System_message}
    Please classify the following text and provide your answer as either "machine generated" or "human written".

    Text: {data_point["statement"]}
    label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

X_test = X_test.sample(frac=0.2, random_state=42)  # random_state for reproducibility

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'status']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])



/tmp/ipykernel_30/859203623.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
/tmp/ipykernel_30/859203623.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [20]:
X_train.status.value_counts(),X_eval.status.value_counts()

(status
 machine    3922
 human      2360
 Name: count, dtype: int64,
 status
 machine    483
 human      302
 Name: count, dtype: int64)

In [21]:
y_true.value_counts()

status
machine    165
human       71
Name: count, dtype: int64

In [22]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [23]:
train_data['text'][3]

'You are a highly skilled AI model specialized in detecting whether a given text is machine-generated or human-written. Your task is to analyze the text provided below in English and determine its origin.\n    Please classify the following text and provide your answer as either "machine generated" or "human written".\n\n    Text: The development of fully autonomous vehicles has been an ongoing pursuit in the automotive industry with significant advancements in recent years. Self-driving cars have the potential to revolutionize transportation by increasing road safety, improving efficiency, and enhancing accessibility. However, there are still limitations to current technology that pose significant challenges to achieving a completely driverless car.\n\nOne of the main limitations is the reliance on smart roads and supportive infrastructure. For a fully autonomous vehicle to navigate safely, advanced sensors and communication systems are needed to interact with smart roads and traffic m

## Loading the model and tokenizer

In [12]:
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

## Model evalution before fine-tuning

In [24]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["human", "machine"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=20, 
                        temperature=0.4)
        
        result = pipe(prompt)
#         print(result)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")
    
    return y_pred

In [25]:
def evaluate(y_true, y_pred):
    labels = ["human", "machine"]
    mapping = {label: idx for idx, label in enumerate(labels)}
    
    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:

y_pred = predict(X_test, model, tokenizer)
print(y_pred)
evaluate(y_true, y_pred)

  0%|          | 0/236 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  4%|▍         | 10/236 [02:59<1:03:06, 16.76s/it]

## Extracting the linear modules names

In [27]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [28]:
modules = find_all_linear_names(model)
modules

['v_proj', 'down_proj', 'gate_proj', 'o_proj', 'up_proj', 'q_proj', 'k_proj']

## Setting up the model

In [29]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    save_total_limit=2,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    eval_strategy="steps",                    # save checkpoint every epoch
    eval_steps = 0.1
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pass

Map:   0%|          | 0/6282 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

## Model Training

In [30]:
print("Training started")

# Train model
trainer.train()

print("Training ended")


Training started


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[at

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
wandb.finish()
model.config.use_cache = True

## Saving the model and tokenizer

In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
import shutil
import os

# Define your variables
dir_to_zip = output_dir

# Define the name of the output zip file
last_word = model.split('/')[-1]
output_zip = f"mullin_{output_dir}.zip"

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', dir_to_zip)
print(f"Zipped contents of {dir_to_zip} into {output_zip}")

## Testing model after fine-tuning 

In [ ]:
sub_df = pd.read_json('/kaggle/input/coling-25-task-1/en_devtest_text_id_only.jsonl', lines=True)
sub_df

def generate_test_prompt_sub(data_point):
    return f"""
    {System_message}
    Please classify the following text and provide your answer as either "machine generated" or "human written".
    
    Text: {data_point["text"]}
    label: """.strip()

sub_df['text'] = sub_df.apply(generate_test_prompt_sub, axis=1)
sub_df = sub_df.sample(frac=0.01, random_state=42)  # random_state for reproducibility

print(len(sub_df))

predictions = predict(sub_df, model, tokenizer)

print("prediction ended")

prediction_file_taska = 'task_a_llm_after_training.jsonl' 
predictions_df = pd.DataFrame({'id': sub_df.id, 'label': predictions})
predictions_df.to_json(prediction_file_taska, lines=True, orient='records')
predictions_df

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)